# Deep Learning Tutorial - Modeling

In [1]:
import numpy as np
import pprint

In [2]:
INPUT_SIZE = 2
LAYER_SIZES = [5, 5, 1]
LAYER_SIZES.insert(0, INPUT_SIZE)

In [3]:
def initialize_network():
    architecture = {}
    for i, size_index in enumerate(range(1, len(LAYER_SIZES))):
        architecture[f'layer_{i+1}'] = {
            'w': np.random.randn(LAYER_SIZES[size_index],
                                 LAYER_SIZES[size_index-1]) * 0.1,
            'b': np.random.randn(LAYER_SIZES[size_index])
        }
    return architecture

In [4]:
network = initialize_network()
pprint.pprint(network)

{'layer_1': {'b': array([ 0.684286  , -0.45235335, -0.72462178,  1.10932135,  1.16686128]),
             'w': array([[ 0.15311693,  0.02071988],
       [-0.13693191, -0.10549693],
       [-0.00387081,  0.20481531],
       [-0.097135  , -0.04318543],
       [ 0.02156991, -0.01516054]])},
 'layer_2': {'b': array([ 1.63754842,  0.21767353,  1.12040649, -0.99304295, -1.81855364]),
             'w': array([[ 0.01833445, -0.03805091,  0.04265919,  0.05733997, -0.0165144 ],
       [-0.2192589 ,  0.07136384, -0.02729625,  0.08657769, -0.04503367],
       [-0.03927704, -0.02913509,  0.10876017, -0.09403395,  0.03146266],
       [ 0.04609964,  0.11874883, -0.0605361 , -0.09544864, -0.09379869],
       [-0.11340498,  0.12389206,  0.08107716, -0.10272101, -0.00741991]])},
 'layer_3': {'b': array([-0.41068865]),
             'w': array([[-0.10454984, -0.04219592, -0.09553875, -0.01056738, -0.00060624]])}}


In [5]:
def sigmoid_activation(Z):
    activation = 1/(1 + np.exp(-1*Z))
    return activation

In [6]:
def relu_activation(Z):
    activation = np.max(0, Z)
    return activation

In [7]:
def dZ_sigmoid(dA, Z):
    sigmoid = sigmoid_activation(Z)
    dZ = dA * sigmoid * (1 - sigmoid)
    return dZ

In [8]:
def dZ_relu(dA, Z):
    dZ = np.copy(dA)
    dZ[Z <= 0] = 0
    return dZ

In [9]:
act_map = {
    'sigmoid': sigmoid_activation,
    'relu': relu_activation
}

In [10]:
dZ_map = {
    'sigmoid': dZ_sigmoid,
    'relu': dZ_relu
}

In [11]:
def single_forward_pass(A_previous, W, b, activation):
    try:
        act_function = act_map[activation]
    except KeyError:
        print(f'The activation {activation} is not recognized.\nIt must be one of the following: {list(act_map.keys())}')
        return None
    
    Z = np.dot(W, A_previous) + b
    A = act_function(Z)
    
    return A, Z

In [11]:
def full_forward_pass():
    return None

In [12]:
def cross_entropy_cost_function():
    return None

In [13]:
def single_backward_pass():
    return None

In [14]:
def full_backward_pass():
    return None

In [15]:
def train_nn():
    return None

### Resources
This notebook has been inspired by the Towards Data Science post [Let’s code a Neural Network in plain NumPy](https://towardsdatascience.com/lets-code-a-neural-network-in-plain-numpy-ae7e74410795).